使用函数式 API 完成以下功能：撰写一篇文章并中断以请求人工审核

- 此工作流将撰写一篇关于“猫”这一主题的文章，然后暂停以获取人工审核。
- 工作流可以无限期中断，直到提供审核结果为止。
- 当工作流恢复时，它会从最开始执行，但由于 write_essay 任务的结果已经保存，因此会从检查点加载任务结果，而不是重新计算。

In [3]:
import time
import uuid

from langgraph.func import entrypoint, task
from langgraph.types import interrupt
from langgraph.checkpoint.memory import MemorySaver

@task
def write_essay(topic: str) -> str:
    """Write an essay about the given topic."""
    time.sleep(3) # This is a placeholder for a long-running task.
    return f"An essay about topic: {topic}"

@entrypoint(checkpointer=MemorySaver())
def workflow(topic: str) -> dict:
    """A simple workflow that writes an essay and asks for a review."""
    # 要获取任务的结果，你可以同步等待（使用 result()），也可以异步等待（使用 await）
    essay = write_essay("cat").result()
    is_approved = interrupt({
        # Any json-serializable payload provided to interrupt as argument.
        # It will be surfaced on the client side as an Interrupt when streaming data
        # from the workflow.
        "essay": essay, # The essay we want reviewed.
        # We can add any additional information that we need.
        # For example, introduce a key called "action" with some instructions.
        "action": "Please approve/reject the essay",
    })

    return {
        "essay": essay, # The essay that was generated
        "is_approved": is_approved, # Response from HIL
    }


config = {
    "configurable": {
        "thread_id": str(uuid.uuid4())
    }
}

for item in workflow.stream("cat", config):
    print(item)

{'write_essay': 'An essay about topic: cat'}
{'__interrupt__': (Interrupt(value={'essay': 'An essay about topic: cat', 'action': 'Please approve/reject the essay'}, resumable=True, ns=['workflow:b37b3818-ef7d-7ff5-3110-e85bbf4da50e']),)}


In [4]:
from langgraph.types import Command

# Get review from a user (e.g., via a UI)
# In this case, we're using a bool, but this can be any json-serializable value.
human_review = True

for item in workflow.stream(Command(resume=human_review), config):
    print(item)

{'workflow': {'essay': 'An essay about topic: cat', 'is_approved': True}}
